In [89]:
from collections import namedtuple, Counter
import csv

import requests

In [67]:
Record = namedtuple('Record', 'zonecode, shapelen, public_land, objectid, proffer, shapearea, zonetype, cluster, jurisdiction')

In [83]:
def get_fairfax_data():
    data = []
        
    url = 'http://data-fairfaxcountygis.opendata.arcgis.com/datasets/9e111a7ccbc54e7db10ece2d8d215e54_1.csv'
    r = requests.get(url)
    txt = r.text.split('\n')
    csv_txt = csv.DictReader(txt)
    for row in csv_txt:
        record = parse_row(row)
        data.append(record)
    return data

In [84]:
def parse_row(row):
    new_row = {}
    new_row['zonecode'] = row['ZONECODE']
    new_row['shapelen'] = float(row['SHAPELEN'])
    new_row['public_land'] = row['PUBLIC_LAND']
    new_row['objectid'] = int(row['\ufeffOBJECTID'])
    new_row['proffer'] = row['PROFFER']
    new_row['shapearea'] = float(row['SHAPEAREA'])
    new_row['zonetype'] = row['ZONETYPE']
    new_row['cluster'] = row['CLUSTER_']
    new_row['jurisdiction'] = row['JURISDICTION']
    
    return Record(**new_row)

In [86]:
data = get_fairfax_data()

In [88]:
len(data)

6358

In [93]:
Counter((item.public_land for item in data))

Counter({'NO': 4511, 'YES': 1847})

In [98]:
Counter((item.zonetype for item in data))

Counter({'COMMERCIAL': 923,
         'INDUSTRIAL': 387,
         'OTHER': 24,
         'PLANNED UNITS': 1038,
         'RESIDENTIAL': 3961,
         'TYSON': 25})

In [102]:
# 5 largest items by shape area
def get_largest_area():
    return sorted(data, key=lambda x: -x.shapearea)[:5]

In [103]:
get_largest_area()

[Record(zonecode='R-C', shapelen=80092.5400189624, public_land='NO', objectid=25079, proffer='', shapearea=144685951.569454, zonetype='RESIDENTIAL', cluster='', jurisdiction='FAIRFAX COUNTY'),
 Record(zonecode='R-E', shapelen=53687.0539938699, public_land='NO', objectid=48439, proffer='', shapearea=143184984.163796, zonetype='RESIDENTIAL', cluster='', jurisdiction='FAIRFAX COUNTY'),
 Record(zonecode='R-C', shapelen=119120.840614294, public_land='NO', objectid=47481, proffer='', shapearea=134986339.185122, zonetype='RESIDENTIAL', cluster='', jurisdiction='FAIRFAX COUNTY'),
 Record(zonecode='R-E', shapelen=110353.95055672, public_land='YES', objectid=33724, proffer='', shapearea=134736934.325036, zonetype='RESIDENTIAL', cluster='', jurisdiction='FAIRFAX COUNTY'),
 Record(zonecode='R-C', shapelen=94860.2070202976, public_land='YES', objectid=33735, proffer='', shapearea=130270541.169266, zonetype='RESIDENTIAL', cluster='', jurisdiction='FAIRFAX COUNTY')]

In [104]:
# 5 smallest by area
def get_smallest_area():
    return sorted(data, key=lambda x: x.shapearea)[:5]

In [105]:
get_smallest_area()

[Record(zonecode='PDH-4', shapelen=7.03911927272365, public_land='NO', objectid=28609, proffer='', shapearea=0.0952385, zonetype='PLANNED UNITS', cluster='', jurisdiction='FAIRFAX COUNTY'),
 Record(zonecode='PDH-4', shapelen=7.17512728016114, public_land='NO', objectid=28603, proffer='', shapearea=0.107457, zonetype='PLANNED UNITS', cluster='', jurisdiction='FAIRFAX COUNTY'),
 Record(zonecode='R-1', shapelen=54.708138844327, public_land='NO', objectid=30609, proffer='NO', shapearea=0.2645675, zonetype='RESIDENTIAL', cluster='', jurisdiction='FAIRFAX COUNTY'),
 Record(zonecode='PDH-4', shapelen=14.9931559814692, public_land='NO', objectid=28620, proffer='', shapearea=0.733865, zonetype='PLANNED UNITS', cluster='', jurisdiction='FAIRFAX COUNTY'),
 Record(zonecode='PDH-4', shapelen=13.2463026174713, public_land='NO', objectid=12148, proffer='', shapearea=0.921855, zonetype='PLANNED UNITS', cluster='', jurisdiction='FAIRFAX COUNTY')]

In [125]:
def calc_avg_area_per_unit(zone):
    zone_data = [record for record in data if record.zonecode == zone]
    total_area = sum([record.shapearea for record in zone_data])
    return int(total_area / len(zone_data))

In [134]:
def calc_largest_zone_avg():
    zones = {}
    for zone in set([record.zonecode for record in data]):
        zones[zone] = calc_avg_area_per_unit(zone)
    return sorted(zones.items(), key=lambda x: -x[1])[:5]

In [135]:
zones = calc_largest_zone_avg()

In [136]:
zones

[('R-C', 10410371),
 ('R-E', 9583128),
 ('PRC', 5670206),
 ('O&LI', 4511419),
 ('PD-W', 4025568)]

In [137]:
[record for record in data if record.zonecode == 'R-C']

[Record(zonecode='R-C', shapelen=626.392962946458, public_land='NO', objectid=25072, proffer='', shapearea=21649.672296, zonetype='RESIDENTIAL', cluster='', jurisdiction='FAIRFAX COUNTY'),
 Record(zonecode='R-C', shapelen=5444.06948917249, public_land='NO', objectid=25073, proffer='', shapearea=1352008.316768, zonetype='RESIDENTIAL', cluster='', jurisdiction='FAIRFAX COUNTY'),
 Record(zonecode='R-C', shapelen=239.376399692304, public_land='NO', objectid=25074, proffer='', shapearea=1638.071848, zonetype='RESIDENTIAL', cluster='', jurisdiction='FAIRFAX COUNTY'),
 Record(zonecode='R-C', shapelen=186.705853960043, public_land='NO', objectid=25075, proffer='', shapearea=747.91701, zonetype='RESIDENTIAL', cluster='', jurisdiction='FAIRFAX COUNTY'),
 Record(zonecode='R-C', shapelen=174.045530376792, public_land='NO', objectid=25076, proffer='', shapearea=1414.31086, zonetype='RESIDENTIAL', cluster='', jurisdiction='FAIRFAX COUNTY'),
 Record(zonecode='R-C', shapelen=1800.8419291544, public_la